In [3]:
import pandas as pd
import urllib as url
import numpy as np
import json
import requests
import sys

# Load table of locations and Ids's
Ref = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv',
                    dtype={'WaterDischarge': str, 'PHLevel': str,'WaterTemp': str})
sites = Ref.USGS_ID.tolist()
string = ''

for i in sites:
    string = string+','+str(i)
sites=string[1:]

# Metrics to load from stations
# 00060 = discharge
# 00400 = PH level
# 00010 = water temperature
parameters = '00060'+','+'00400'+','+'00010'

# URL to load json of data 
# Use format=rmd for readable URL
link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+
        sites+'&period=P20D&parameterCd='+
        parameters)
r=requests.get(link)
data = json.loads(r.text)

#Create empty DataFrame with columns
water = pd.DataFrame(columns=['Name','VariableDescription','Value','DateTime'])

i=0
for items in data['value']['timeSeries']:
    for name in items['sourceInfo']:
        if name == 'siteName':
            # Station name
            names = items['sourceInfo'][name]
    for name in items:
         if name == 'variable':
            # Parameter Description 
            var = items[name]['variableDescription']
    for values in items['values']:
        for v in values['value']:
            # Append values for each column
            water.loc[i] = [names,var,v['value'],v['dateTime']]
            i=1+i


# Break apart Date and Time
dates = [date[0:10] for date in water.DateTime]
times = [time[11:16] for time in water.DateTime]
water['DateTime'] = pd.to_datetime(pd.Series(dates)+' '+pd.Series(times))
water.Value = water.Value.astype(float)
waterpp = water.pivot_table(
                    index='DateTime',
                    columns=['VariableDescription','Name'],
                    values=['Value'])
water.VariableDescription.loc[
    water.VariableDescription == 'Discharge, cubic feet per second'] = 'Discharge'
water.VariableDescription.loc[
    water.VariableDescription == 'Temperature, water, degrees Celsius'] = 'Temperature'
water.VariableDescription.loc[
    water.VariableDescription =='pH, water, unfiltered, field, standard units' ] = 'pH'
water = water.sort_values(['Name','VariableDescription','DateTime'])

dts = pd.Series(pd.to_datetime(water.DateTime.astype(np.int64).groupby(np.arange(len(water))//16).mean()))
vals = pd.Series((water.Value.groupby(np.arange(len(water))//16).mean()))
ind = pd.Series((pd.Series(water.index).groupby(np.arange(len(water))//16).mean()).astype(int))
df = pd.concat([dts,vals,ind],axis=1)
info = water[['Name','VariableDescription']]
df = pd.merge(df,info,
              left_on=df[0],
              right_on=info.index,how='left')
df = df.drop(columns=['key_0',0])

full_df = pd.read_csv('CurrentWater.csv')
full_df = full_df.append(df)
full_df = full_df[['DateTime','Value','Name','VariableDescription']]
full_df.drop_duplicates(inplace=True)
full_df[['DateTime','Value','Name','VariableDescription']].to_csv('CurrentWater.csv',index=False)

C:\Users\Scott\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [2]:
# https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
# https://www.ncdc.noaa.gov/cdo-web/webservices/v2#data
import sys
import requests
import pandas as pd
import json
from datetime import datetime,date, timedelta

Token = 'ziLDJecqBlvTSNGOzOmPPLmBZtXSfBwR'

referencedata = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')
referencedata['ZipCode2'] = referencedata['ZipCode'].fillna(0).astype(int)
today = date.today()
Dateslist = [today - timedelta(days = day) for day in range(100)]
startdate = min(Dateslist)
enddate = max(Dateslist)
print(startdate,enddate)
all_data = pd.read_csv('CurrentPrecip.csv',index_col=0).drop_duplicates()

for row in range(0,len(referencedata)):
    print(referencedata.Name[row],' Zip Codes')
    name = referencedata.Name[row]
    zipcode = referencedata.ZipCode[row]
    zipcode2 = referencedata.ZipCode2[row]
    print(zipcode,' ',zipcode2)


    data_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&units=metric&limit=100&locationid=ZIP:'+str(zipcode)+'&startdate='+str(startdate)+'&enddate='+str(enddate)
    
    r = requests.get(data_url, headers={'token':Token})
    d = json.loads(r.text)

    # print(json.dumps(d, indent=4, sort_keys=True))
 
    if len(d) == 0:
        data_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=100&locationid=ZIP:'+str(zipcode2)+'&startdate='+str(startdate)+'&enddate='+str(enddate)
        r = requests.get(data_url, headers={'token':Token})
        d = json.loads(r.text)

    dates = []
    precips = []

    #get all items in the response which are precipitation readings
    try:
        precip = [item for item in d['results'] if item['datatype']=='PRCP']
        precips += [item['value'] for item in precip]
        dates += [item['date'] for item in precip]
    except KeyError:
        print('No data available at this Zip Code: ',zipcode2)
        continue

    df = pd.DataFrame()
    df['Date'] = [datetime.strptime(d, '%Y-%m-%dT%H:%M:%S') for d in dates]
    if len(precips) > 0:
        df['Precip'] = [v for v in precips]

    df['Name'] = name
    all_data = all_data.append(df,ignore_index=True)
# print(all_data)
all_data.to_csv('C:/Users/Scott/Desktop/Projects/River_Data/CurrentPrecip.csv')

2021-12-06 2022-03-15
Russian River HaciendaBridge  Zip Codes
95436   95436
Navarro River  Zip Codes
95432   95432
Noyo River  Zip Codes
95437   95437
San Lorenzo River Upper  Zip Codes
95018   95018
Mad River  Zip Codes
95573   95573
Putah Creek  Zip Codes
95694   95694
Truckee River Farad  Zip Codes
96161   96161
Little Truckee River  Zip Codes
96161   96161
Smith River CC  Zip Codes
95531   95531
Smith River Fort Dick  Zip Codes
95531   95531
Eel River Fort Seward  Zip Codes
95542   95542
Eel River Scotia  Zip Codes
95565   95565
Eel River Fernbridge  Zip Codes
95540   95540
San Lorenzo River  Zip Codes
95018   95018
Hat Creek  Zip Codes
96013   96013
Yuba River  Zip Codes
95944   95944
North Fork American River  Zip Codes
95603   95603
Pit River  Zip Codes
96051   96051
East Walker  Zip Codes
93517   93517
East Fork Carson  Zip Codes
96120   96120
Trinity River Lewiston  Zip Codes
96052   96052
Trinity River Junction City  Zip Codes
96048   96048
Rogue River Grants Pass  Zip Codes


In [5]:

data_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=100&locationid=ZIP:'+str(zipcode2)+'&startdate='+'2022-03-01'+'&enddate='+'2022-03-14'#str(enddate)
r = requests.get(data_url, headers={'token':Token})
d = json.loads(r.text)
d


{'metadata': {'resultset': {'offset': 1, 'count': 6, 'limit': 100}},
 'results': [{'date': '2022-03-01T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1CAYB0003',
   'attributes': ',,N,',
   'value': 0},
  {'date': '2022-03-02T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1CAYB0003',
   'attributes': ',,N,',
   'value': 0},
  {'date': '2022-03-03T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1CAYB0003',
   'attributes': ',,N,',
   'value': 0},
  {'date': '2022-03-04T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1CAYB0003',
   'attributes': ',,N,',
   'value': 3},
  {'date': '2022-03-05T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1CAYB0003',
   'attributes': ',,N,',
   'value': 3},
  {'date': '2022-03-06T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1CAYB0003',
   'attributes': ',,N,',
   'value': 5}]}

In [3]:
import re
import pandas as pd
import numpy as np
import requests
import json
import time
import datetime as dt

# Parameters
rivers_reference = pd.read_csv('RiverReferenceTable.csv')

sites = rivers_reference.USGS_ID.tolist()
string = ''

for i in sites:
    string = string+','+str(i)
sites=string[1:]

# URL to load json of data 
# Use format=rmd for readable URL
link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+sites)
r=requests.get(link)
Geodata = json.loads(r.text)

Forecast_df = pd.DataFrame()

# Gather information from API
for i in range(len(Geodata['value']['timeSeries'])):
    lat = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['latitude'])
    long = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['longitude'])
    baseStation = Geodata['value']['timeSeries'][i]['sourceInfo']['siteName']
    # Search API for Lat Long Position
    r = requests.get('https://api.weather.gov/points/'+lat+','+long)
    json_data = r.json()

    # Retrieve Forecast for Lat Long Position
    forecast_json = requests.get(json_data['properties']['forecast']).json()
    try: 
        if forecast_json['status'] == 500:
            continue
    except:
        KeyError
    forecast = forecast_json['properties']['periods']
    Forecast_df_temp = pd.DataFrame(forecast).drop(columns='number')
    Forecast_df_temp['Lat'] = lat
    Forecast_df_temp['Long'] = long

    Forecast_df_temp['Station'] = baseStation
    Forecast_df = Forecast_df.append(Forecast_df_temp,ignore_index=True)

# Cleaning DataFrame
df = Forecast_df.drop_duplicates()
df.startTime = pd.to_datetime(df.startTime,utc=True)
df.endTime = pd.to_datetime(df.endTime,utc=True)
df.drop(columns=['temperatureTrend','icon'],inplace=True)

# Functions for Rain types
def LightRain(s):
    if re.search('Slight Chance Rain',s):
        value = 1
    elif re.search('Light Rain',s):
        value = 1  
    else: value = 0
    return value

def Rain(s):
    if re.search('Rain Showers',s):
        value = 1
    else: value = 0
    return value

def HeavyRain(s):
    if re.search('Heavy Rain',s):
        value = 1
    else: value = 0
    return value

def WindTranslate(s):
    s = ' '+s
    l = len(s)-4
    value = s[l-2:l]
    return int(value)
    
# Create Rain Boolean Features
df['LightRain'] = df.shortForecast.apply(lambda x: LightRain(x))
df['Rain'] = df.shortForecast.apply(lambda x: Rain(x))
df['HeavyRain'] = df.shortForecast.apply(lambda x: HeavyRain(x))
df['WindMph'] = df.windSpeed.apply(lambda x: WindTranslate(x))

# Create final Dataframe with Key
WeatherForecast = df[['Station','name','startTime','endTime','temperature','windDirection','WindMph','LightRain','Rain','HeavyRain']]
WeatherForecast = WeatherForecast.set_index('Station').join(rivers_reference[['USGS Name','Name']].set_index('USGS Name'))
WeatherForecast = WeatherForecast.reset_index().rename(columns={'index':'Station','Name':'StationName'})
WeatherForecast['Date_Name'] = WeatherForecast.startTime.dt.date.astype(str)+'_'+WeatherForecast.StationName

# WeatherForecast.to_csv('WeatherForecast.csv',index=False)
full_df = pd.read_csv('WeatherForecast.csv')
full_df.append(WeatherForecast).drop_duplicates()[['Station','name','startTime','endTime','temperature','windDirection','WindMph','LightRain','Rain','HeavyRain','StationName','Date_Name']].to_csv('WeatherForecast.csv')

<ipython-input-3-aacbe4dc33ac>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LightRain'] = df.shortForecast.apply(lambda x: LightRain(x))
<ipython-input-3-aacbe4dc33ac>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rain'] = df.shortForecast.apply(lambda x: Rain(x))
<ipython-input-3-aacbe4dc33ac>:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://